In [1]:
import pandas as pd

from pathlib import Path
#import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## IEX Constants
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [3]:
token_status = test_token ## Set to either real token or test token for IEX

In [4]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [5]:
## Load IEX to get ETF statistics 
def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [6]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

In [7]:
def get_float_data_FMP(symbol):
    url_float_shares = fmpbase_urlv4+'shares_float?symbol='
    url_query_float_data = url_float_shares+symbol+'&apikey='+api_key
    resp_data = requests.get(url_query_float_data)
    #df = pd.DataFrame(resp_data.json())
    json_ = resp_data.json()
    return json_[0]

def get_company_profile_FMP_json(symbol):
    ## https://fmpcloud.io/api/v3/profile/AAPL?apikey='yourkeyhere'
    url_company_profile_url = fmpbase_urlv3+'profile/'+symbol+'?apikey='+api_key
    resp_data = requests.get(url_company_profile_url)
    json_response = resp_data.json()
    return json_response[0]

In [8]:
symbol_success_list = load_obj('../Resources/02_symbol_success_list.pkl')
len(symbol_success_list)

6620

In [9]:
## Check if duplicates 
symbol_success_list_set = set(symbol_success_list) 
contains_duplicates = len(symbol_success_list) != len(symbol_success_list_set)
print(contains_duplicates)

True


In [10]:
if contains_duplicates == True:
    symbol_success_list = list( dict.fromkeys(symbol_success_list) )
    print(len(symbol_success_list))

5493


In [11]:
# test_symbol = symbol_success_list[0]
# test_symbol

In [12]:
# path = '../FilesExport_TimeSeries_DFs/'+test_symbol+'_combined_df.pkl'
# test_load_df = load_obj(path)
# df = test_load_df[0][test_symbol]
# df.sort_index()

In [14]:
# ## If you need key directly from imported pkl obj
# for key in test_load_df[0]:
#     print(key)

In [15]:
## Create while loop that accesses APIs
## The point of creating the while loop is to be able to pick up where you left off, 
## in case the for loop fails, for example if the computer loses connection to the APIs, 
## the code will fail. Can then load and view the exported list, and pick up where the 
## for loop left off by editing the x and y starting values 


## Initialize lists for successful calls
datadict_success_list = []    ## Contains symbols, used to track successful api calls. 
                              ## Updates after each run of for loop.  

test_list = symbol_success_list[0:100]
    
length_ = len(symbol_success_list)
test_length = len(test_list)

In [17]:
iex_token = test_token     ## Token status for IEX - use real_token or test_token variables 
loop_length = test_length  ## Set to length_ to run as full, or your test_length

x = 0    
y = 50
increment = 50  ## Make sure increment is right

In [18]:
while x < loop_length:
    
    ## Set y value for end of while loop
    if y >= loop_length: y = loop_length
    ## In theory this should stop the for loop from crashing 
    ## at the end and running out of index range 

    range_var = range(x,y)
    ## Create/Clear temp list 
    temp_symbol_list = []

    for i in range_var:
        symbol = symbol_success_list[i]
        
        ## Add to temp list 
        temp_symbol_list.append(symbol)
        
        
        err_symbol = symbol    ## Helps for error tracking if loop fails

        path = '../FilesExport_TimeSeries_DFs/'+symbol+'_combined_df.pkl'
        import_obj = load_obj(path)
        time_series_df = import_obj[0][symbol].sort_index()

        ## IEX 
        try:
            iex_stats = get_IEX_statistics(stock_ticker=symbol,token=iex_token)        
            sharesOutstanding = iex_stats['sharesOutstanding']
            peRatio = iex_stats['peRatio']
            beta = iex_stats['beta']
            week52high = iex_stats['week52high']
            week52low = iex_stats['week52low']
            week52change = iex_stats['week52change']
            avg10Volume = iex_stats['avg10Volume']
            avg30Volume = iex_stats['avg30Volume']
            marketcap_IEX = iex_stats['marketcap']
        except:
            iex_stats = 0
            sharesOutstanding = 0
            peRatio = 0
            beta = 0
            week52high = 0
            week52low = 0
            week52change = 0
            avg10Volume = 0
            avg30Volume = 0
            marketcap_IEX = 0

        ## FMP 
        try:
            fmp_profile = get_company_profile_FMP_json(symbol)
            exchange = fmp_profile['exchangeShortName']
            marketcap_FMP = fmp_profile['mktCap']
        except:
            fmp_profile = 0
            exchange = 0
            marketcap_FMP = 0
        try:
            float_data = get_float_data_FMP(symbol)
            floatShares = float_data['floatShares']
        except:
            float_data = 0
            floatShares = 0
            
        ## Create data_dict:
        data = {symbol : {
        'Fundamentals':                    			    ## Fundamental Data about the stock 
        {
            'sharesOutstanding': sharesOutstanding,		## Sourced from IEX for ETFs, FMP for Equity
            'floatShares': floatShares,			        ## Can't source for ETFs at the moment - can get from FMP for Equity 
            'exchange': exchange,                       ## Sourced from FMP 
            'final_close_price': time_series_df.iloc[-1]['close'],		## Sourced from time_series_df, use last close price 
            'peRatio': peRatio,			                ## Sourced from iex_statistics, =0 for ETFs
            'beta': beta,			                    ## Sourced from iex_statistics for Equity, =0 for ETFs
            'week52high': week52high,					## Sourced from iex_statistics
            'week52low': week52low,					    ## Sourced from iex_statistics
            'week52change': week52change,               ## Sourced from iex_statistics
            'avg10Volume': avg10Volume,				    ## Sourced from iex_statistics
            'avg30Volume': avg30Volume,				    ## Sourced from iex_statistics
            'marketcap_IEX': marketcap_IEX,		        ## Sourced from iex_statistics
            'marketcap_FMP': marketcap_FMP			    ## Sourced from FMP	
        },
        'dataFrame':time_series_df,							## FMP historical merged with Nasdaq Short Data and SEC FTD Data
        'companyProfile':fmp_profile,         	## Sourced from FMP, otherwise =0
        'floatData':float_data,   					## Sourced from FMP, otherwise =0
        #'textNews':['article1','article2','article3'],    	## Not sourced
        #'returns':'returns_data',    						## Not calculated
        'iex_statistics': iex_stats     	## Added to completed data_dicts, both ETFs and Equity
        }}

        export_path = Path('../FilesExport_DataDicts/'+symbol+'_data_dict.pkl')
        save_obj(data,export_path)
        
        ## End for loop 
        
    ## If for loop successful, extend temp_symbol lits to main datadict_success_list 
    datadict_success_list.extend(temp_symbol_list)
    
    ## Export main list each time, and rewrite during each iteration of while loop 
    ## Can read in list afterwards, to figure out where loop went wrong, and where to restart 
    save_obj(datadict_success_list,'../Resources/03_datadict_success_list.pkl')
    
    ## Check before run, if incorrect, can waste a lot of API credits  
    x += increment
    y += increment

In [19]:
## Test import 

import_obj = load_obj('../FilesExport_DataDicts/AAA_data_dict.pkl')

In [20]:
import_obj

{'AAA': {'Fundamentals': {'sharesOutstanding': 408082,
   'floatShares': 0,
   'exchange': 'AMEX',
   'final_close_price': 25.02,
   'peRatio': 0,
   'beta': -0.001070470930587948,
   'week52high': 26.18,
   'week52low': 25,
   'week52change': 0.004356621615075882,
   'avg10Volume': 2220,
   'avg30Volume': 2055,
   'marketcap_IEX': 10121134,
   'marketcap_FMP': 0},
  'dataFrame':                     open          high           low         close  \
  Date                                                                 
  2016-01-04   9474.320313   9782.429688   9474.320313   9705.400391   
  2016-01-05   9551.349609  10475.700195   9551.349609  10398.599609   
  2016-01-06  10398.599609  10475.700195  10013.500000  10167.599609   
  2016-01-07  10090.500000  10398.599609   9936.480469  10167.599609   
  2016-01-08  10013.500000  10090.500000   9782.429688  10090.500000   
  ...                  ...           ...           ...           ...   
  2021-12-27     25.015000     25.015000   